In [1]:
import numpy as np
import pandas as pd
import glob

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/rich/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
article_files = glob.glob('data/inferred_topics/articles_*.csv')

# Topic numbers that are Politics, Business, and Healthcare
# topics_nums = [2, 19, 22, 23, 27, 43, 44, 45, 47, 
#             49, 52, 57, 60, 63, 67, 82, 84, 
#             85, 87, 88, 90, 95, 98]

for file in article_files:
    #read in the topic inferred articles
    df_topics = pd.read_csv(file)

    #filter rows to only the relevant topics (Politics, Business, and Healthcare)
    # df_topics = df.loc[df['topic_id'].isin(topics_nums)]
    
    #Need to bring in the article text. Going to join on the 'id' column
    #First change the file name and path
    article_file = file.replace('inferred_topics', 'original_articles').replace('csv', 'jl')
    df_article = pd.read_json(article_file, lines=True)
    df_article = df_article.dropna()
    df_article['word_count'] = df_article['article'].str.split().str.len()

    df_all = pd.merge(df_topics, df_article, left_on='id', right_on='id', how='inner')

    #Apply sentiment analysis. Create 'sent_scores' column and populate
    df_all['sent_scores'] = df_all['article'].apply(lambda article: sid.polarity_scores(article))

    #Create 'compound' column and populate
    df_all['compound']  = df_all['sent_scores'].apply(lambda score_dict: score_dict['compound'])

    df_all['comp_score'] = df_all['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')


    df_all.drop(['Unnamed: 0', 'date_y', 'url', 'article', 'title', 'category'], axis=1, inplace=True)

    #writing the file

    filepath = file.replace('data/inferred_topics/articles_', 'data/output/topic_sent_output_')
    df_all.to_csv(filepath, index=False)


KeyboardInterrupt: 